In [972]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [973]:
import MaxQuant_Postprocessing_Functions as mq
import re
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [974]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# Classify using Protein Data
* Use proteinGroups.txt outputted from MaxQuant
* SVC variations
* K neighbors
* Decision tree

** With train-test split, 0.4 test:**

Algorithm | Accuracy Score
:-----:|:-----:
SVC kernel = linear | 1.0
LinearSVC | 0.75
SVC kernel = rbf | 0.167
SVC kernel = poly | 1.0
KNN | 1.0
Decision Tree | 0.67

In [975]:
#########################
#
# Load and clean mouse data
#
#########################

file = "D:\proteinGroups.txt"

df = mq.load_df(file)
df = mq.clean_weakly_identified(df)
df = mq.remove_dup_proteinIDs(df)

iBAQ_df = mq.slice_by_column(df, 'protein', 'iBAQ ')

# Rename columns so that all instances "before" string are replaced with "after" string
def rename_columns(df, before, after):
    columns = df.columns.values.tolist()
    new_columns = []
    for column in columns:
        new_column = re.sub(before, after, column)
        new_columns.append(new_column)
        
    return new_columns

iBAQ_df.columns = rename_columns(iBAQ_df, 'Adult', 'Mouse')

groups = ['Brain', 'Heart', 'Kidney', 'Liver', 'Lung']
organ_columns = {} # 'Liver': ['iBAQ 04_Liver', 'iBAQ 05_Liver', ...]
organ_counts = {} # 'Liver': 
    
iBAQ_df = mq.filter_low_observed(iBAQ_df, groups, organ_columns, organ_counts)

In [976]:
#########################
#
# Normalize data and impute missing values with data frame minimum/2
#
#########################

mq.log2_normalize(iBAQ_df)
mq.median_normalize(iBAQ_df)

iBAQ_df['Majority protein IDs'] = iBAQ_df['Majority protein IDs'].str[:-6] # strip off '_Mouse'
iBAQ_df.set_index('Majority protein IDs', inplace = True)
iBAQ_df = mq.impute_missing(iBAQ_df)

C:\Users\kush494\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4451: RuntimeWarning: divide by zero encountered in log2
  return lib.map_infer(x.asobject, func)


In [977]:
#########################
#
# Map each column name to a corresponding label
#
#########################

"""
Args: 
    df (dataframe)
    columns (list of strings): list of all column names in df
    organ_to_columns (dict): mapping of each organ to its column names {str: list of str}
    
Returns: 
    List of strings representing the labels for each dataframe column
"""
def get_labels(df, columns, organ_to_columns):
    labels = []

    for column in columns:
        key = next(key for key, value in organ_to_columns.items() if column in value)
        labels.append(key)
        
    return labels

In [978]:
iBAQ_df = iBAQ_df[organ_columns['Brain'] + organ_columns['Heart'] + organ_columns['Kidney'] + organ_columns['Liver'] + organ_columns['Lung']]

columns = iBAQ_df.columns.values.tolist()
col_labels = get_labels(iBAQ_df, columns, organ_columns)

print(columns)
print(col_labels)

['iBAQ Mouse_07_Brain', 'iBAQ Mouse_08_Brain', 'iBAQ Mouse_09_Brain', 'iBAQ Mouse_10_Brain', 'iBAQ Mouse_11_Brain', 'iBAQ Mouse_12_Brain', 'iBAQ Mouse_07_Heart', 'iBAQ Mouse_08_Heart', 'iBAQ Mouse_09_Heart', 'iBAQ Mouse_10_Heart', 'iBAQ Mouse_11_Heart', 'iBAQ Mouse_12_Heart', 'iBAQ Mouse_07_Kidney', 'iBAQ Mouse_08_Kidney', 'iBAQ Mouse_09_Kidney', 'iBAQ Mouse_10_Kidney', 'iBAQ Mouse_11_Kidney', 'iBAQ Mouse_12_Kidney', 'iBAQ Mouse_04_Liver', 'iBAQ Mouse_05_Liver', 'iBAQ Mouse_06_Liver', 'iBAQ Mouse_07_Liver', 'iBAQ Mouse_08_Liver', 'iBAQ Mouse_09_Liver', 'iBAQ Mouse_07_Lung', 'iBAQ Mouse_08_Lung', 'iBAQ Mouse_09_Lung', 'iBAQ Mouse_10_Lung', 'iBAQ Mouse_11_Lung', 'iBAQ Mouse_12_Lung']
['Brain', 'Brain', 'Brain', 'Brain', 'Brain', 'Brain', 'Heart', 'Heart', 'Heart', 'Heart', 'Heart', 'Heart', 'Kidney', 'Kidney', 'Kidney', 'Kidney', 'Kidney', 'Kidney', 'Liver', 'Liver', 'Liver', 'Liver', 'Liver', 'Liver', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung']


In [979]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import preprocessing

# Transpose so that proteins are columns (components)
# Scale data
scaled_data = preprocessing.scale(iBAQ_df.T)

#########################
#
# Split data and labels into test and train groups
#
#########################
X_train, X_test, y_train, y_test = cross_validation.train_test_split(iBAQ_df.T, col_labels, test_size=0.2, random_state=0, stratify=col_labels)

"""
X_train = np.concatenate([scaled_data[:4, :], scaled_data[6:10, :], scaled_data[12:16, :], scaled_data[18:22, :], scaled_data[24:28, :], scaled_data[30:34, :]], axis=0)

X_test = np.concatenate([scaled_data[4:6, :], scaled_data[10:12, :], scaled_data[16:18, :], scaled_data[22:24, :], scaled_data[28:30, :], scaled_data[34:, :]], axis=0)

y_train = labels[:4] + labels[6:10] + labels[12:16] + labels[18:22] + labels[24:28] + labels[30:34]
y_test = labels[4:6] + labels[10:12] + labels[16:18] + labels[22:24] + labels[28:30] + labels[34:]
"""
print(X_train.shape)
print(X_test.shape)

(24, 4399)
(6, 4399)


In [980]:
#########################
#
# Draw PCA graph of data
#
#########################

import pandas as pd
import matplotlib.pyplot as plt

base_dir = 'D:\\Images\\Classifier\\'
color_mapping = mq.map_colors(groups, organ_columns)
columns = iBAQ_df.columns.values.tolist()

pca, pca_data = mq.do_pca(iBAQ_df.copy())
per_var, labels = mq.make_scree_plot(pca, base_dir)
mq.draw_pca_graph(columns, pca_data, base_dir, color_mapping, per_var, labels)

In [981]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_t_train = pca.transform(X_train)
X_t_test = pca.transform(X_test)

print(X_t_train.shape)
print(X_t_test.shape)
print(y_train)

(24, 2)
(6, 2)
['Kidney', 'Kidney', 'Kidney', 'Lung', 'Brain', 'Heart', 'Heart', 'Liver', 'Lung', 'Brain', 'Liver', 'Kidney', 'Heart', 'Brain', 'Lung', 'Heart', 'Kidney', 'Liver', 'Brain', 'Liver', 'Brain', 'Lung', 'Liver', 'Heart']


## SVC Variations

In [982]:
#########################
#
# Basic SVC Classification with train-test split
#
#########################

from sklearn.metrics import accuracy_score

clf = SVC(C=1)
clf.fit(X_t_train, y_train)
y_pred = clf.predict(X_t_test)

print('score', accuracy_score(y_pred, y_test))
print('pred label', clf.predict(X_t_test))
print('actual', y_test)

score 0.166666666667
pred label ['Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart']
actual ['Liver', 'Kidney', 'Heart', 'Lung', 'Lung', 'Brain']


In [983]:
#########################
#
# Basic SVC Classification with cross-validation
#
#########################
clf2 = SVC(C=1)
scores = cross_val_score(clf2, iBAQ_df.T, col_labels, cv=4)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.8  0.9  1.   0.4]
Accuracy: 0.78 (+/- 0.46)


In [984]:
from sklearn.svm import LinearSVC

#########################
#
# SVC variations
#
#########################

def try_SVC_models(X_train, y_train, X_test, y_test):
    C = 1.0  # SVM regularization parameter
    models = (SVC(kernel='linear', C=C),
              LinearSVC(C=C),
              SVC(kernel='rbf', gamma=0.7, C=C),
              SVC(kernel='poly', degree=3, C=C))

    # Fit all the models
    models = (clf.fit(X_train, y_train) for clf in models)

    for model in models:
        model_y_pred = model.predict(X_test)
        print('\n*** Model: ', model, '\n')
        print('score', accuracy_score(model_y_pred, y_test))
        print('pred label', model_y_pred)
        print('actual', y_test)

In [985]:
try_SVC_models(X_t_train, y_train, X_t_test, y_test)


*** Model:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False) 

score 1.0
pred label ['Liver' 'Kidney' 'Heart' 'Lung' 'Lung' 'Brain']
actual ['Liver', 'Kidney', 'Heart', 'Lung', 'Lung', 'Brain']

*** Model:  LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0) 

score 0.833333333333
pred label ['Kidney' 'Kidney' 'Heart' 'Lung' 'Lung' 'Brain']
actual ['Liver', 'Kidney', 'Heart', 'Lung', 'Lung', 'Brain']

*** Model:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.7, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False) 

score 0.16666666

## K Neighbors

In [986]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_t_train, y_train)
y_pred = knn.predict(X_t_test)

print('score', accuracy_score(y_pred, y_test))
print('pred', y_pred)
print('actual', y_test)

score 1.0
pred ['Liver' 'Kidney' 'Heart' 'Lung' 'Lung' 'Brain']
actual ['Liver', 'Kidney', 'Heart', 'Lung', 'Lung', 'Brain']


In [996]:
knn2 = KNeighborsClassifier()
scores = cross_val_score(knn2, iBAQ_df.T, col_labels, cv=4)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

knn2.fit(iBAQ_df.T, col_labels)

[ 1.  1.  1.  1.]
Accuracy: 1.00 (+/- 0.00)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

## Decision Tree

In [988]:
decision_tree_clf = tree.DecisionTreeClassifier()
decision_tree_clf = decision_tree_clf.fit(X_t_train, y_train)
dt_pred = decision_tree_clf.predict(X_t_test)

print('score', accuracy_score(dt_pred, y_test))
print('pred', dt_pred)
print('actual', y_test)

score 1.0
pred ['Liver' 'Kidney' 'Heart' 'Lung' 'Lung' 'Brain']
actual ['Liver', 'Kidney', 'Heart', 'Lung', 'Lung', 'Brain']


In [989]:
dt2 = tree.DecisionTreeClassifier()
scores = cross_val_score(dt2, iBAQ_df.T, col_labels, cv=4)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 1.   0.9  1.   1. ]
Accuracy: 0.97 (+/- 0.09)


# Testing classification of human lung
* Human dataset info:
    * Instrument: QExactHF03
    * Separation Type: LC-Waters-Formic_3hr
    * Tool: MSGFPlus_MzMl
    * Jobs: 1498824-1498852
    * Param file: MSGFDB_PartTryp_MetOx_StatCysAlk_10ppmParTol.txt
    * Unlabelled samples

* Map mouse proteins to corresponding human proteins
* Load Human lung data
* Join dataframes on proteinID and only keep human data for testing
* Run human lung data through classifier trained on mouse tissue data 

In [990]:
#########################
#
# Load and clean mouse-human correspondance data
#
#########################

### Load correspondance data
import pandas as pd

mapping_file = r'D:\Human_Mouse_Mapping.txt'
mapping_df = pd.read_csv(mapping_file, usecols=['Matched Term', 'Symbol', 'Species'], sep='\t', lineterminator='\r', encoding = 'latin1')
mapping_df = mapping_df.replace(r'\n','', regex=True)

### Filter out entries not containing human in the "Species" column
mapping_df = mapping_df[mapping_df['Species'].isnull() | mapping_df['Species'].str.contains('Human')]
mapping_df.set_index('Matched Term', inplace=True)
mapping_df.drop(['Species'], axis=1, inplace=True)

mapping_df['Symbol'].replace(to_replace=' (includes others)', value='', inplace=True) # remove trailing comments
print(mapping_df.head())

             Symbol
Matched Term       
ABL1_MOUSE     ABL1
AKT1_MOUSE     AKT1
HEM2_MOUSE     ALAD
HEM0_MOUSE    ALAS2
A4_MOUSE        APP


In [991]:
#########################
#
# Load and clean human data
#
#########################

human_lung_file = r'F:\Human_Lung_Raw_Files\LungMAP\combined\txt\human_lung_proteinGroups.txt'
human_groups = ['Human_Lung']

human_lung_df = mq.load_df(human_lung_file)
human_lung_df = mq.clean_weakly_identified(human_lung_df)
human_lung_df = mq.remove_dup_proteinIDs(human_lung_df)
        
human_lung_iBAQ_df = mq.slice_by_column(human_lung_df, 'protein', 'iBAQ ') 
    
human_lung_organ_columns = {}
human_lung_organ_counts = {} 
    
human_lung_iBAQ_df = mq.filter_low_observed(human_lung_iBAQ_df, human_groups, human_lung_organ_columns, human_lung_organ_counts)

human_lung_iBAQ_df['Majority protein IDs'] = human_lung_iBAQ_df['Majority protein IDs'].str[:-6]
mq.log2_normalize(human_lung_iBAQ_df)
mq.median_normalize(human_lung_iBAQ_df)

human_lung_iBAQ_df.set_index('Majority protein IDs', inplace = True)
human_lung_iBAQ_df = mq.impute_missing(human_lung_iBAQ_df)

C:\Users\kush494\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4451: RuntimeWarning: divide by zero encountered in log2
  return lib.map_infer(x.asobject, func)


In [992]:
#########################
#
# Change mouse proteinIDs to common symbol
#
#########################

mouse_proteins = iBAQ_df.index.values.tolist()
human_proteins = human_lung_iBAQ_df.index.values.tolist()
raw_mappings = mapping_df.to_dict('index') # {mouse protein: {'Symbol': common protein}}
mappings = {}

# Break up rows with multiple mouse proteins
for old_key, val in raw_mappings.items():
    keys = old_key.split()
    for new_key in keys:
        mappings[new_key] = raw_mappings[old_key]
        
iBAQ_df.reset_index(inplace=True)

for protein in mouse_proteins:
    if protein not in human_proteins:
        to_replace = protein + '_MOUSE'
        if to_replace in mappings:
            mapping = mappings[to_replace]
            new_sym = mapping['Symbol']
            #if(protein != new_sym):
                #print('Found one!', to_replace, '-->', new_sym)
            iBAQ_df.replace(protein, new_sym, inplace=True)
        
iBAQ_df.set_index('Majority protein IDs', inplace=True)
        

In [993]:
#########################
#
# Join mouse data to human data so that human_t_test has same shape as trained data
#
#########################

combined_df = iBAQ_df.join(human_lung_iBAQ_df)
combined_df = mq.impute_missing(combined_df)
all_columns = combined_df.columns.values.tolist()

combined_pca, combined_pca_data = mq.do_pca(combined_df)

# Add mapping of 'Human_Lung' to all human lung columns, and new organ group to groups
organ_columns.update(human_lung_organ_columns)
groups.append('Human_Lung')

# Draw scree plot of all human and mouse tissues
combined_dir = base_dir + 'Human_'
combined_color_mapping = mq.map_colors(groups, organ_columns)

combined_per_var, combined_labels = mq.make_scree_plot(combined_pca, combined_dir)
mq.draw_pca_graph(all_columns, combined_pca_data, combined_dir, combined_color_mapping, combined_per_var, combined_labels)

combined_df.drop(combined_df.columns[:30], axis=1, inplace=True) # Drop mouse data
print(combined_df.head())

                      iBAQ Human_Lung_001  iBAQ Human_Lung_002  \
Majority protein IDs                                             
1433B                           26.495533            27.139241   
1433E                           27.406570            28.235220   
1433F                           24.655306            25.732509   
1433G                           25.598352            26.609912   
1433S                           23.849066            24.211665   

                      iBAQ Human_Lung_004  iBAQ Human_Lung_005  \
Majority protein IDs                                             
1433B                           27.413408            27.553327   
1433E                           28.483613            28.573078   
1433F                           25.452296            24.832387   
1433G                           26.463207            26.265243   
1433S                           24.303326            24.792482   

                      iBAQ Human_Lung_007  iBAQ Human_Lung_008  \
Majority

## K Nearest Neighbors

In [994]:
scaled_human_data = preprocessing.scale(combined_df.T)
human_t_test = pca.transform(scaled_human_data)
human_y_test = ['Lung' for x in range(29)]

human_pred = knn.predict(human_t_test)

print('score', accuracy_score(human_pred, human_y_test))
print('pred', human_pred)
print('actual', human_y_test)

score 0.0
pred ['Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart']
actual ['Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung']


C:\Users\kush494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [997]:
human_pred_2 = knn2.predict(combined_df.T)

print('score', accuracy_score(human_pred_2, human_y_test))
print('pred', human_pred_2)
print('actual', human_y_test)

score 0.0
pred ['Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart']
actual ['Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung']


## Decision Tree

In [748]:
human_dt_pred = decision_tree_clf.predict(human_t_test)

print('score', accuracy_score(human_dt_pred, human_y_test))
print('pred', human_dt_pred)
print('actual', human_y_test)

score 0.0
pred ['Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart' 'Heart'
 'Heart' 'Heart']
actual ['Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung']


## Random Forest

## Naive Bayes/LDA

# Classify using Peptide Data
* Use peptides.txt output from MaxQuant
* SVC varations
* K nearest neighbors
* Decision tree

Algorithm | Accuracy Score
:-----:|:-----:
SVC kernel = linear | 1.0
LinearSVC | 1.0
SVC kernel = rbf | 0.167
SVC kernel = poly | 0.917
KNN | 1.0
Decision Tree | 1.0

In [749]:
#########################
#
# Load and clean peptide data
#
#########################

peptide_file = "D:\peptides.txt"

peptide_df = mq.load_df(peptide_file)
peptide_df = mq.slice_by_column(peptide_df, 'peptide', 'LFQ')
peptide_df.columns = rename_columns(peptide_df, 'Adult', 'Mouse')

peptide_organ_columns = {}
peptide_organ_counts = {}
peptide_df = mq.filter_low_observed(peptide_df, groups, peptide_organ_columns, peptide_organ_counts)
mq.log2_normalize(peptide_df)
mq.median_normalize(peptide_df)

peptide_df = peptide_df.replace(r'\n','', regex=True)
peptide_df.set_index('Sequence', inplace = True)
peptide_df = mq.impute_missing(peptide_df)

peptide_columns = peptide_df.columns.values.tolist()
peptide_labels = get_labels(peptide_df, peptide_columns, peptide_organ_columns)
print(peptide_labels)

C:\Users\kush494\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4451: RuntimeWarning: divide by zero encountered in log2
  return lib.map_infer(x.asobject, func)


['Liver', 'Liver', 'Liver', 'Brain', 'Heart', 'Kidney', 'Liver', 'Lung', 'Brain', 'Heart', 'Kidney', 'Liver', 'Lung', 'Brain', 'Heart', 'Kidney', 'Liver', 'Lung', 'Brain', 'Heart', 'Kidney', 'Lung', 'Brain', 'Heart', 'Kidney', 'Lung', 'Brain', 'Heart', 'Kidney', 'Lung']


In [750]:
#########################
#
# Split data and labels into test and train groups
#
#########################

scaled_peptide_data = preprocessing.scale(peptide_df.T)

### Randomly split:
peptide_X_train, peptide_X_test, peptide_y_train, peptide_y_test = cross_validation.train_test_split(peptide_df.T, peptide_labels, test_size=0.4, random_state=0, stratify=peptide_labels)

peptide_pca = PCA(n_components=4)
peptide_pca.fit(peptide_X_train)
peptide_X_t_train = peptide_pca.transform(peptide_X_train)
peptide_X_t_test = peptide_pca.transform(peptide_X_test)

print(peptide_X_t_train.shape)
print(peptide_X_t_test.shape)
print(peptide_y_train)

(18, 4)
(12, 4)
['Lung', 'Liver', 'Heart', 'Heart', 'Kidney', 'Liver', 'Heart', 'Kidney', 'Kidney', 'Liver', 'Brain', 'Kidney', 'Brain', 'Brain', 'Brain', 'Lung', 'Lung', 'Heart']


In [751]:
#########################
#
# Draw PCA plots for peptide data
#
#########################

peptide_dir = base_dir + 'Mouse_Peptide_'
peptide_color_mapping = mq.map_colors(groups, peptide_organ_columns)
columns = peptide_df.columns.values.tolist()

peptide_pca, peptide_pca_data = mq.do_pca(peptide_df.copy())
peptide_per_var, peptide_labels = mq.make_scree_plot(peptide_pca, peptide_dir)
mq.draw_pca_graph(columns, peptide_pca_data, peptide_dir, peptide_color_mapping, peptide_per_var, peptide_labels)


In [752]:
#########################
#
# Attempt to classify using SVC algorithm
#
#########################

peptide_clf = SVC()
peptide_clf.fit(peptide_X_t_train, peptide_y_train)
peptide_y_pred = peptide_clf.predict(peptide_X_t_test)

print('score', accuracy_score(peptide_y_pred, peptide_y_test))
print('pred label', peptide_y_pred)
print('actual', peptide_y_test)

score 0.166666666667
pred label ['Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney'
 'Kidney' 'Kidney' 'Kidney' 'Kidney']
actual ['Brain', 'Lung', 'Lung', 'Kidney', 'Heart', 'Liver', 'Liver', 'Heart', 'Liver', 'Kidney', 'Lung', 'Brain']


In [753]:
#########################
#
# Attempt to classify using 4 SVC variations 
#
#########################

try_SVC_models(peptide_X_t_train, peptide_y_train, peptide_X_t_test, peptide_y_test)


*** Model:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False) 

score 1.0
pred label ['Brain' 'Lung' 'Lung' 'Kidney' 'Heart' 'Liver' 'Liver' 'Heart' 'Liver'
 'Kidney' 'Lung' 'Brain']
actual ['Brain', 'Lung', 'Lung', 'Kidney', 'Heart', 'Liver', 'Liver', 'Heart', 'Liver', 'Kidney', 'Lung', 'Brain']

*** Model:  LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0) 

score 1.0
pred label ['Brain' 'Lung' 'Lung' 'Kidney' 'Heart' 'Liver' 'Liver' 'Heart' 'Liver'
 'Kidney' 'Lung' 'Brain']
actual ['Brain', 'Lung', 'Lung', 'Kidney', 'Heart', 'Liver', 'Liver', 'Heart', 'Liver', 'Kidney', 'Lung', 'Brain']

*** Model:  SVC(C=1.0, cache_size=200, class_weight=None, 

In [754]:
#########################
#
# Attempt to classify using K Neighbors algorithm
#
#########################

peptide_knn = KNeighborsClassifier()
peptide_knn.fit(peptide_X_t_train, peptide_y_train)
peptide_y_pred = peptide_knn.predict(peptide_X_t_test)

print('score', accuracy_score(peptide_y_pred, peptide_y_test))
print('pred', peptide_y_pred)
print('actual', peptide_y_test)

score 1.0
pred ['Brain' 'Lung' 'Lung' 'Kidney' 'Heart' 'Liver' 'Liver' 'Heart' 'Liver'
 'Kidney' 'Lung' 'Brain']
actual ['Brain', 'Lung', 'Lung', 'Kidney', 'Heart', 'Liver', 'Liver', 'Heart', 'Liver', 'Kidney', 'Lung', 'Brain']


In [755]:
#########################
#
# Attempt to classify using decision tree
#
#########################

peptide_dt_clf = tree.DecisionTreeClassifier()
peptide_dt_clf = peptide_dt_clf.fit(peptide_X_t_train, peptide_y_train)
peptide_dt_pred = peptide_dt_clf.predict(peptide_X_t_test)

print('score', accuracy_score(peptide_dt_pred, peptide_y_test))
print('pred', peptide_dt_pred)
print('actual', peptide_y_test)

score 1.0
pred ['Brain' 'Lung' 'Lung' 'Kidney' 'Heart' 'Liver' 'Liver' 'Heart' 'Liver'
 'Kidney' 'Lung' 'Brain']
actual ['Brain', 'Lung', 'Lung', 'Kidney', 'Heart', 'Liver', 'Liver', 'Heart', 'Liver', 'Kidney', 'Lung', 'Brain']
